<h2>--- Day 4: Lobby ---</h2>

Jeremy Bloom solutions for  [Advent of Code](https://adventofcode.com): [2025 Day 4](https://adventofcode.com/2025/day/4)


In [284]:
# output file
def get_input_file(filename):
  try:
    with open(filename, 'r') as f:
      content = f.read()
      #print(content)
      content = content.strip()
      contents_as_list = content.split("\n")
  except FileNotFoundError:
    print("no such file found")
  return contents_as_list

day4_test = get_input_file("day4_example.txt")

In [285]:
day4_test

['..@@.@@@@.',
 '@@@.@.@.@@',
 '@@@@@.@.@@',
 '@.@@@@..@.',
 '@@.@@@@.@@',
 '.@@@@@@@.@',
 '.@.@.@.@@@',
 '@.@@@.@@@@',
 '.@@@@@@@@.',
 '@.@.@@@.@.']

### notes
- @ is a roll of paper
- @ can be accessed if FEWER than four of the eight adj positions are occupied
- a wall is considered OPEN

## needs refactoring
this code is far more complicated than it needs to be, but good enough for now as it's understandable and shields the complexity from the user

In [304]:
class AdventGrid:
  VAL_PAPER = "@"
  VAL_EMPTY = "."
  VAL_PAPER_REMOVED = "x"
  VAL_OFFGRID = "-"
  
  def __init__(self, grid_data):
    """
    grid_data is a list of lists|rows
    let's call the bottom row = row 1, but note zero based indicing in use
    let's call the first col = col 1, ...

    values:  VAL_*
    """
    self.grid_data = grid_data.copy() # !important - otherwise grid tied to grid_data input
    self.num_rows = len(grid_data)  
    self.num_cols = len(grid_data[0])
    self.space_needed_to_move = 5 # 3 or less of the 8 spots can be blocked and paper still moved

    # verify grid_data
    for r in range(self.num_rows):
      for c in range(self.num_cols):     
        if self.get_val_by_row_col(r+1,c+1) not in (self.VAL_PAPER, self.VAL_EMPTY, self.VAL_PAPER_REMOVED):
          print(f"error at row {r+1} col {c+1}: {self.get_val_by_row_col(r+1,c+1)}")

  def print(self):
    for r in range(self.num_rows):
      print(self.grid_data[r])

  def print_full(self):
    for r in range(self.num_rows):
      # print(f"row {r + 1:3}: {self.grid_data[r]}")
      m = f"{r + 1:3}: "
      for i in range( len(self.grid_data[r] )):
        if i % 5 == 0:
          m += "  "
        m += "{:2s}".format(self.grid_data[r][i])
      print(m)
                      
        
  def get_row_by_num(self, row_num):
    # starts at row 1
    if row_num <= 0 or row_num > self.num_rows:
      return
    return self.grid_data[row_num - 1]
  
  def get_val_by_row_col(self, row_num, col_num):
    #print(f"get_val_by_row_col({row_num},{col_num})...", end="")
    # starts at row 1 col 1
    if row_num <= 0 or row_num > self.num_rows or col_num <= 0 or col_num > self.num_cols:
      #print("not in grid")
      return self.VAL_OFFGRID
    else:
      #print(f"self.grid_data[row_num-1][col_num-1]")
      return self.grid_data[row_num-1][col_num-1]


  def set_val_by_row_col(self, row_num, col_num, new_val):
    # starts at row 1 col 1
    if row_num <= 0 or row_num > self.num_rows or col_num <= 0 or col_num > self.num_cols:
      print(f"error - cannot set a val by invalid row_num{row_num}   col_num {col_num}")
      return False
    else:
      # need to get the whole row and replace it
      row_val = self.grid_data[row_num - 1]
      #print(f"row_val    : {row_val}")
      row_val_new = ""
      for i in range(len(row_val)):
        if i == col_num - 1:
          row_val_new += new_val
        else:
          row_val_new += row_val[i]
        #print(f"row_val_new: {row_val_new}")
      
      self.grid_data[row_num - 1] = row_val_new
      return True
        
  ######

 
  def is_removable_paper_at_row_col(self,row_num, col_num):
    """
    returns True iff
    - there is PAPER at this spot
    - there are no more than max_blocks other papers surrounding this spot
    """

    
    # starts at row 1
    if row_num <= 0 or row_num > self.num_rows:
      print(f"error - invalid row_num {row_num}")
      return False
    if col_num <= 0 or col_num > self.num_cols:
      print(f"error - invalid col_num {col_num}")
      return False

    if self.get_val_by_row_col(row_num, col_num) != self.VAL_PAPER:
      # there is no paper here, so can't remove
      return False
    
    # get the eight neighbors - start at 12n, clockwise
    V = []
    V.append( self.get_val_by_row_col(row_num-1, col_num  ) )
    V.append( self.get_val_by_row_col(row_num-1, col_num+1) )
    V.append( self.get_val_by_row_col(row_num  , col_num+1) )
    V.append( self.get_val_by_row_col(row_num+1, col_num+1) )
    V.append( self.get_val_by_row_col(row_num+1, col_num  ) )
    V.append( self.get_val_by_row_col(row_num+1, col_num-1) )
    V.append( self.get_val_by_row_col(row_num  , col_num-1) )
    V.append( self.get_val_by_row_col(row_num-1, col_num-1) )

    num_paper   = V.count(self.VAL_PAPER)
    num_empty   = V.count(self.VAL_EMPTY)
    num_offgrid = V.count(self.VAL_OFFGRID)
    num_paper_removed =  V.count(self.VAL_PAPER_REMOVED)

    if (num_paper + num_empty + num_offgrid + num_paper_removed) != 8:
      print("error - unexpected return val", num_paper, num_empty, num_offgrid, num_paper_removed)
      return False

    if num_empty + num_offgrid >= self.space_needed_to_move:
      # room to move
      return True
    else:
      # not enough space to remove
      #print(f"is_removable_paper_at_row_col({row_num},{col_num}) False as no space")
      return False

  
  def remove_paper_at_row_col(self,row_num, col_num, warn=0):
    if not self.is_removable_paper_at_row_col(row_num, col_num):
      if warn:
        print("error - cannot remove paper at this location")
      return False
    else:
      return self.set_val_by_row_col(row_num, col_num, self.VAL_PAPER_REMOVED) 



  def remove_rolls(self):
    for r in range(self.num_rows):
      for c in range(self.num_cols):
        if self.is_removable_paper_at_row_col(r+1, c+1):
          self.remove_paper_at_row_col(r+1, c+1)


  def count_movables(self):
    n = 0
    for r in range(self.num_rows):
      for c in range(self.num_cols):
        if self.get_val_by_row_col(r+1,c+1) == self.VAL_PAPER_REMOVED:
          n += 1
    return n



In [305]:
#day4_test = get_input_file("day4_example.txt")
g = AdventGrid(day4_test)

#g.print()
g.print_full()
g.remove_rolls()
g.print()
print(g.count_movables())


  1:   . . @ @ .   @ @ @ @ . 
  2:   @ @ @ . @   . @ . @ @ 
  3:   @ @ @ @ @   . @ . @ @ 
  4:   @ . @ @ @   @ . . @ . 
  5:   @ @ . @ @   @ @ . @ @ 
  6:   . @ @ @ @   @ @ @ . @ 
  7:   . @ . @ .   @ . @ @ @ 
  8:   @ . @ @ @   . @ @ @ @ 
  9:   . @ @ @ @   @ @ @ @ . 
 10:   @ . @ . @   @ @ . @ . 
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
13


In [311]:
day4_input = get_input_file("day4_input.txt")
g = AdventGrid(day4_input)
print(g.num_rows, g.num_cols)

136 136


In [315]:
g.remove_rolls()
print(g.count_movables())


1428
